<a href="https://colab.research.google.com/github/julianovale/simulacao_python/blob/master/0008_ex_trem_kronecker_artigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import I, Matrix, symbols, Symbol, eye
from datetime import datetime
import numpy as np
import pandas as pd


In [ ]:
'''
Rotas
'''
R1 = Matrix([[0,"L1p3",0,0,0,0],[0,0,"L1v1",0,0,0],[0,0,0,"L1p4",0,0],[0,0,0,0,"L1v3",0],[0,0,0,0,"L1v4",0],[0,0,0,0,0,0]])
R2 = Matrix([[0,"L2p3",0,0,0,0],[0,0,"L2v2",0,0,0],[0,0,0,"L2p5",0,0],[0,0,0,0,"L2v3",0],[0,0,0,0,"L2v5",0],[0,0,0,0,0,0]])
R3 = Matrix([[0,"L3p3",0,0,0,0],[0,0,"L3v5",0,0,0],[0,0,0,"L3p1",0,0],[0,0,0,0,"L3v3",0],[0,0,0,0,"L3v1",0],[0,0,0,0,0,0]])

In [ ]:
'''
Seções de bloqueio
'''
T1 = Matrix([[0, "T1p1"],["T1v1", 0]])
T2 = Matrix([[0, "T2p2"],["T2v2", 0]])
T3 = Matrix([[0, "T3p3"],["T3v3", 0]])
T4 = Matrix([[0, "T4p4"],["T4v4", 0]])
T5 = Matrix([[0, "T5p5"],["T5v5", 0]])

In [ ]:
momento_inicio = datetime.now() 
'''
Algebra
'''
rotas = np.kron(np.kron(R1, eye(6))+np.kron(eye(6), R2), eye(6))+np.kron(eye(36),R3)
secoes = np.kron(np.kron(np.kron((np.kron(T1, eye(2)) + np.kron(eye(2), T2)),eye(2))+np.kron(eye(4),T3), eye(2))+np.kron(eye(8), T4), eye(2))+np.kron(eye(16),T5)
sistema = np.kron(rotas, secoes)

# calcula tempo de processamento
tempo_processamento = datetime.now() - momento_inicio

In [ ]:
sistema = pd.DataFrame(data=sistema,index=list(range(1,np.size(sistema,0)+1)), columns=list(range(1,np.size(sistema,1)+1)))

In [ ]:
sistema.shape

(6912, 6912)

In [ ]:
print(tempo_processamento)

0:00:46.958837


In [ ]:
sistema

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6873,6874,6875,6876,6877,6878,6879,6880,6881,6882,6883,6884,6885,6886,6887,6888,6889,6890,6891,6892,6893,6894,6895,6896,6897,6898,6899,6900,6901,6902,6903,6904,6905,6906,6907,6908,6909,6910,6911,6912
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,L3p3,L3p3,0,L3p3,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,L3p3,0,0,L3p3,0,L3p3,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,L3p3,0,0,L3p3,0,0,L3p3,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,L3p3,L3p3,0,0,0,0,L3p3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,L3p3,0,0,0,0,L3p3,L3p3,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6909,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6910,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
colunas = ['denode', 'paranode', 'aresta']
grafo = pd.DataFrame(columns=colunas)
r = 1
c = 1
for j in range(np.size(sistema,0)):
  for i in range(np.size(sistema,0)):
    if sistema.loc[r,c]==0 and c < np.size(sistema,0):
      c += 1
    elif c < np.size(sistema,0):
      grafo.loc[len(grafo)+1] = (r, c, sistema.loc[r,c])  
      c += 1
    else:
      c = 1
      r += 1

In [ ]:
grafo

,denode,paranode,aresta
1,1,34,L3p3
2,1,35,L3p3
3,1,37,L3p3
4,1,41,L3p3
5,1,49,L3p3
...,...,...,...
83676,6880,6864,L3v1
83677,6880,6872,L3v1
83678,6880,6876,L3v1
83679,6880,6878,L3v1


In [ ]:
grafo.to_csv('grafo.csv', sep=";")

In [ ]:
from google.colab import files
files.download('grafo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>